# Manchester Borough Price Change Map

This notebook loads `2019 to today manc.csv`, calculates average house prices for 2024 and 2025 for each Greater Manchester borough,
and displays a map with borough markers coloured green for an increase in price and red for a decrease.

In [ ]:
# Install pandas and folium if needed
import sys, subprocess
for pkg in ['pandas', 'folium']:
    try:
        __import__(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])


In [ ]:
import pandas as pd
import folium

# Approximate borough centre coordinates
borough_coords = {
    'BOLTON': (53.5769, -2.4283),
    'BURY': (53.5933, -2.2990),
    'MANCHESTER': (53.4808, -2.2426),
    'OLDHAM': (53.5405, -2.1183),
    'ROCHDALE': (53.6160, -2.1550),
    'SALFORD': (53.4890, -2.2901),
    'STOCKPORT': (53.4084, -2.1493),
    'TAMESIDE': (53.4800, -2.0800),
    'TRAFFORD': (53.4280, -2.2920),
    'WIGAN': (53.5450, -2.6370)
}

cols = ['transaction_id','price','date','postcode','property_type','old_new','duration','paon','saon','street','locality','town','district','county','ppd_cat','record']
df = pd.read_csv('2019 to today manc.csv', names=cols, header=None)

df['price'] = df['price'].astype(float)
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year

avg = df.groupby(['district','year'])['price'].mean().unstack()
change = avg[2025] - avg[2024]

map_center = [53.4808, -2.2426]
m = folium.Map(location=map_center, zoom_start=10)
for borough, diff in change.dropna().items():
    lat, lon = borough_coords.get(borough.upper(), map_center)
    colour = 'green' if diff > 0 else 'red'
    folium.CircleMarker(
        location=[lat, lon],
        radius=10,
        color=colour,
        fill=True,
        fill_color=colour,
        popup=f'{borough}: {diff:+.2f}'
    ).add_to(m)

m